<a href="https://colab.research.google.com/github/kat1478/analiza_zadowolenia_z_zycia_badanych/blob/main/analiza_zadowolenia_z_zycia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analiza zadowolenia z życia badanych

## Cel analizy

Celem niniejszej analizy jest zbadanie czynników wpływających na poziom zadowolenia z życia osób badanych w różnych kontekstach demograficznych i społecznych. Badanie obejmuje:

- **Zadowolenie z życia** badane przez cztery kolejne miesiące.
- **Wpływ czynników demograficznych i społecznych**, takich jak:
  - Typ zamieszkania (np. dom, mieszkanie, małe mieszkanie).
  - Status związku (związek formalny, nieformalny, status wolny).
  - Posiadanie zwierzęcia domowego (pies, kot, brak).
- **Predykcję ogólnego poziomu zadowolenia** na podstawie ocen z poszczególnych miesięcy oraz zmiennych społecznych.

## Zakres analizy

W trakcie analizy:
1. Zbadano dane pod kątem różnic w zadowoleniu z życia między różnymi grupami.
2. Przeprowadzono testy statystyczne, takie jak:
   - ANOVA (jedno- i dwuczynnikowa),
   - Testy nieparametryczne (Kruskal-Wallis, Friedman),
   - Testy post-hoc (Tukey, Dwass-Steel-Critchlow-Fligner).
3. Stworzono modele regresji liniowej, aby przewidzieć poziom zadowolenia na podstawie wybranych czynników.
4. Sprawdzono założenia modeli statystycznych, takie jak:
   - Normalność rozkładu reszt,
   - Homogeniczność wariancji,
   - Multikolinearność predyktorów (VIF).

Na koniec podsumowano wyniki i wyciągniemy wnioski, które mogą być pomocne w dalszych badaniach i aplikacjach praktycznych.


##1. Instalacja i import potrzebnych bibliotek

In [56]:
!pip install pyreadstat statsmodels pingouin scikit-posthocs

In [60]:
import pandas as pd
import numpy as np
import pyreadstat
import scikit_posthocs as sp

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats import multitest
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns


##2. Wczytanie pliku .sav

In [13]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/data_science_exam/test2.sav'

df, meta = pyreadstat.read_sav(file_path)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Lp,Wiek,Płeć,Status_związku,Posiadanie_zwierzęcia,Typ_zamieszkania,Liczba_przeczytanych_książek,Liczba_obejrzanych_filmów,Liczba_obejrzanych_odcinków_seriali,Zadowolenie_z_życia_0_do_100,Zadowolenie__styczeń,Zadowolenie__luty,Zadowolenie__marzec,Zadowolenie__kwiecień,Ocena_zadowolenia_po_czterech_miesiącach
0,1.0,młodsi,kobieta,związek nieformalny,kot,1.0,0.0,7.0,15.0,74.0,52.0,48.0,57.0,63.0,58.0
1,2.0,młodsi,mężczyzna,związek formalny,kot,1.0,3.0,10.0,4.0,66.0,43.0,62.0,70.0,61.0,65.0
2,3.0,młodsi,kobieta,status wolny,kot,1.0,0.0,6.0,11.0,62.0,44.0,59.0,72.0,59.0,54.0
3,4.0,młodsi,mężczyzna,związek nieformalny,kot,1.0,0.0,2.0,16.0,72.0,58.0,71.0,51.0,63.0,61.0
4,5.0,młodsi,kobieta,związek formalny,kot,1.0,1.0,8.0,10.0,72.0,60.0,58.0,63.0,75.0,60.0


##3. Wstępna inspekcja danych



In [14]:
df.info()
df.describe()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Lp                                        100 non-null    float64
 1   Wiek                                      100 non-null    object 
 2   Płeć                                      100 non-null    object 
 3   Status_związku                            100 non-null    object 
 4   Posiadanie_zwierzęcia                     100 non-null    object 
 5   Typ_zamieszkania                          100 non-null    float64
 6   Liczba_przeczytanych_książek              100 non-null    float64
 7   Liczba_obejrzanych_filmów                 100 non-null    float64
 8   Liczba_obejrzanych_odcinków_seriali       100 non-null    float64
 9   Zadowolenie_z_życia_0_do_100              100 non-null    float64
 10  Zadowolenie__styczeń                   

,Lp,Wiek,Płeć,Status_związku,Posiadanie_zwierzęcia,Typ_zamieszkania,Liczba_przeczytanych_książek,Liczba_obejrzanych_filmów,Liczba_obejrzanych_odcinków_seriali,Zadowolenie_z_życia_0_do_100,Zadowolenie__styczeń,Zadowolenie__luty,Zadowolenie__marzec,Zadowolenie__kwiecień,Ocena_zadowolenia_po_czterech_miesiącach
0,1.0,młodsi,kobieta,związek nieformalny,kot,1.0,0.0,7.0,15.0,74.0,52.0,48.0,57.0,63.0,58.0
1,2.0,młodsi,mężczyzna,związek formalny,kot,1.0,3.0,10.0,4.0,66.0,43.0,62.0,70.0,61.0,65.0
2,3.0,młodsi,kobieta,status wolny,kot,1.0,0.0,6.0,11.0,62.0,44.0,59.0,72.0,59.0,54.0
3,4.0,młodsi,mężczyzna,związek nieformalny,kot,1.0,0.0,2.0,16.0,72.0,58.0,71.0,51.0,63.0,61.0
4,5.0,młodsi,kobieta,związek formalny,kot,1.0,1.0,8.0,10.0,72.0,60.0,58.0,63.0,75.0,60.0


In [17]:
print("\nUnikalne wartości w kolumnie 'Wiek':")
print(df['Wiek'].unique())

print("\nUnikalne wartości w kolumnie 'Płeć':")
print(df['Płeć'].unique())

print("\nUnikalne wartości w kolumnie 'Status_związku':")
print(df['Status_związku'].unique())

print("\nUnikalne wartości w kolumnie 'Posiadanie_zwierzęcia':")
print(df['Posiadanie_zwierzęcia'].unique())

print("\nUnikalne wartości w kolumnie 'Typ_zamieszkania':")
print(df['Typ_zamieszkania'].unique())


Unikalne wartości w kolumnie 'Wiek':
['młodsi' 'średni wiek' 'starsi']

Unikalne wartości w kolumnie 'Płeć':
['kobieta' 'mężczyzna']

Unikalne wartości w kolumnie 'Status_związku':
['związek nieformalny' 'związek formalny' 'status wolny']

Unikalne wartości w kolumnie 'Posiadanie_zwierzęcia':
['kot' 'pies' 'brak']

Unikalne wartości w kolumnie 'Typ_zamieszkania':
[1. 2. 3.]


##4. Testy normalności, homogeniczności

In [21]:
# Przykład: Zadowolenie_z_życia_0_do_100
shap = pg.normality(df['Zadowolenie_z_życia_0_do_100'].dropna(), method='shapiro')
print(shap)

print("\n=== TEST NORMALNOŚCI (Shapiro-Wilk) - Liczba_obejrzanych_odcinków_seriali ===")
shap_res2 = pg.normality(df['Liczba_obejrzanych_odcinków_seriali'], method='shapiro')
display(shap_res2)


                                     W     pval  normal
Zadowolenie_z_życia_0_do_100  0.981374  0.16987    True

=== TEST NORMALNOŚCI (Shapiro-Wilk) - Liczba_obejrzanych_odcinków_seriali ===


,W,pval,normal
Liczba_obejrzanych_odcinków_seriali,0.989107,0.593002,True


# 5. JEDNOCZYNNIKOWA ANOVA (PYTANIA 3,4,5,6 W TEŚCIE)

In [27]:
print("\n=== ANOVA: Zadowolenie_z_życia_0_do_100 ~ Posiadanie_zwierzęcia ===")
model_a1 = smf.ols('Zadowolenie_z_życia_0_do_100 ~ C(Posiadanie_zwierzęcia)', data=df).fit()
anova_a1 = sm.stats.anova_lm(model_a1, typ=2)
display(anova_a1)
groups = []
for cat in df['Posiadanie_zwierzęcia'].unique():
    subset = df.loc[df['Posiadanie_zwierzęcia'] == cat, 'Zadowolenie_z_życia_0_do_100'].dropna().values
    groups.append(subset)
res_levene = pg.homoscedasticity(groups, method='levene')
print("Wynik homoscedasticity:")
print(res_levene)

stat_lev, p_lev, eq_var = res_levene.values[0]
print(f"Test Levene'a: W={stat_lev:.3f}, p={p_lev:.3f}, equal_var={eq_var}")


=== ANOVA: Zadowolenie_z_życia_0_do_100 ~ Posiadanie_zwierzęcia ===


,sum_sq,df,F,PR(>F)
C(Posiadanie_zwierzęcia),74.467987,2.0,1.648656,0.197651
Residual,2190.692013,97.0,NaN,NaN


Wynik homoscedasticity:
              W      pval  equal_var
levene  0.64733  0.525687       True
Test Levene'a: W=0.647, p=0.526, equal_var=True


In [46]:
print("\n=== ANOVA: Liczba_obejrzanych_odcinków_seriali ~ Status_związku ===")
model_a2 = smf.ols('Liczba_obejrzanych_odcinków_seriali ~ C(Status_związku)', data=df).fit()
anova_a2 = sm.stats.anova_lm(model_a2, typ=2)
display(anova_a2)

# Sprawdzamy normalność reszt i homogeniczność
res2 = model_a2.resid
norm2 = pg.normality(res2, method='shapiro')
print("Normalność reszt (Shapiro):")
display(norm2)

groups2 = []
for cat in df['Status_związku'].unique():
    subset = df.loc[df['Status_związku'] == cat, 'Liczba_obejrzanych_odcinków_seriali'].dropna().values
    groups2.append(subset)
res_levene2 = pg.homoscedasticity(groups2, method='levene')
print("Wynik homoscedasticity:")
print(res_levene2)

stat_lev, p_lev, eq_var = res_levene2.values[0]
print(f"Test Levene'a: W={stat_lev:.3f}, p={p_lev:.3f}, equal_var={eq_var}")
res_levene2 = pg.homoscedasticity(groups2, method='levene')
print("Wynik homoscedasticity:")
print(res_levene2)

stat_lev2 = res_levene2.loc['levene', 'W']
p_lev2    = res_levene2.loc['levene', 'pval']
eq_var2   = res_levene2.loc['levene', 'equal_var']

print(f"Test Levene'a: W={stat_lev2:.3f}, p={p_lev2:.3f}, equal_var={eq_var2}")


=== ANOVA: Liczba_obejrzanych_odcinków_seriali ~ Status_związku ===


,sum_sq,df,F,PR(>F)
C(Status_związku),26.443654,2.0,0.545625,0.581247
Residual,2350.546346,97.0,NaN,NaN


Normalność reszt (Shapiro):


,W,pval,normal
0,0.990758,0.725715,True


Wynik homoscedasticity:
               W      pval  equal_var
levene  0.817372  0.444609       True
Test Levene'a: W=0.817, p=0.445, equal_var=True
Wynik homoscedasticity:
               W      pval  equal_var
levene  0.817372  0.444609       True
Test Levene'a: W=0.817, p=0.445, equal_var=True


In [34]:
# PRZYKŁAD 3: Zadowolenie_z_życia_0_do_100 ~ Wiek (pyt.5,6)
print("\n=== ANOVA: Zadowolenie_z_życia_0_do_100 ~ Wiek ===")
model_a3 = smf.ols('Zadowolenie_z_życia_0_do_100 ~ C(Wiek)', data=df).fit()
anova_a3 = sm.stats.anova_lm(model_a3, typ=2)
display(anova_a3)

# Jeśli ANOVA jest istotna, robimy post-hoc (Tukey).
# Odpowiada to pyt.6: "Który test post-hoc?" i "Jaki wynik?"

posthoc_3 = pairwise_tukeyhsd(
    endog=df['Zadowolenie_z_życia_0_do_100'],
    groups=df['Wiek'],
    alpha=0.05
)
print("\n== Post-hoc Tukey (Zadowolenie_z_życia_0_do_100 vs Wiek) ==")
print(posthoc_3)


=== ANOVA: Zadowolenie_z_życia_0_do_100 ~ Wiek ===


,sum_sq,df,F,PR(>F)
C(Wiek),48.669172,2.0,1.064951,0.34874
Residual,2216.490828,97.0,NaN,NaN



== Post-hoc Tukey (Zadowolenie_z_życia_0_do_100 vs Wiek) ==
  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1    group2   meandiff p-adj   lower  upper  reject
--------------------------------------------------------
młodsi      starsi  -1.0208 0.6541 -3.7815   1.74  False
młodsi średni wiek  -1.6884 0.3225 -4.4713 1.0945  False
starsi średni wiek  -0.6676 0.8401 -3.4905 2.1552  False
--------------------------------------------------------


# 6. TESTY NIEPARAMETRYCZNE (KRUSKAL-WALLIS, PYT.7)

In [47]:
# Mapa do konwersji kategorii na liczby
wiek_map = {
    'młodsi': 1,
    'średni wiek': 2,
    'starsi': 3
}

# Tworzenie nowej kolumny numerycznej dla 'Wiek'
df['Wiek_num'] = df['Wiek'].map(wiek_map)

# Sprawdzenie konwersji
print("\nWartości zmiennej 'Wiek_num':")
print(df[['Wiek', 'Wiek_num']].head())



Wartości zmiennej 'Wiek_num':
     Wiek  Wiek_num
0  młodsi         1
1  młodsi         1
2  młodsi         1
3  młodsi         1
4  młodsi         1


In [52]:
print("\n=== Test Kruskala-Wallisa ===")
print(kw)

print("\nNazwy kolumn w wyniku testu Kruskala-Wallisa:")
print(kw.columns)



=== Test Kruskala-Wallisa ===
                 Source  ddof1         H     p-unc
Kruskal  Status_związku      2  0.029307  0.985453

Nazwy kolumn w wyniku testu Kruskala-Wallisa:
Index(['Source', 'ddof1', 'H', 'p-unc'], dtype='object')


In [53]:
# Zakładając, że kolumna p nazywa się 'p-unc' lub 'pval'
if 'p-unc' in kw.columns:
    p_val = kw['p-unc'][0]
elif 'pval' in kw.columns:
    p_val = kw['pval'][0]
elif 'p-val' in kw.columns:
    p_val = kw['p-val'][0]
else:
    raise KeyError("Nie znaleziono kolumny z wartością p. Sprawdź dostępne kolumny.")

print(f"Test Kruskala-Wallisa: p-value = {p_val}")

Test Kruskala-Wallisa: p-value = 0.985453151596787


<ipython-input-53-e216fd348b4e>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_val = kw['p-unc'][0]


In [58]:
if p_val < 0.05:
    print("\n=== Post-hoc Dwass–Steel–Critchlow–Fligner (DSCF) ===")
    posthoc_kw = sp.posthoc_dscf(df, val_col='Wiek_num', group_col='Status_związku')
    display(posthoc_kw)

    # Znalezienie grup, które nie różnią się (p=1)
    groups_no_diff = posthoc_kw[posthoc_kw == 1]
    print("\nGrupy, które nie różnią się statystycznie pod kątem wieku (p = 1):")
    print(groups_no_diff.dropna(how='all').dropna(axis=1, how='all'))
else:
    print("\nKruskal-Wallis nie jest istotny statystycznie (p >= 0.05). Testy post-hoc nie są konieczne.")



Kruskal-Wallis nie jest istotny statystycznie (p >= 0.05). Testy post-hoc nie są konieczne.


# 7. DWUCZYNNIKOWA ANOVA: Zadowolenie_z_życia_0_do_100 ~ Wiek * Typ_zamieszkania (pyt.8,9,10)

In [37]:
print("\n=== DWUCZYNNIKOWA ANOVA: Zadowolenie_z_życia_0_do_100 ~ Wiek * Typ_zamieszkania ===")
model_2way = smf.ols('Zadowolenie_z_życia_0_do_100 ~ C(Wiek) * C(Typ_zamieszkania)', data=df).fit()
anova_2way = sm.stats.anova_lm(model_2way, typ=2)
display(anova_2way)


=== DWUCZYNNIKOWA ANOVA: Zadowolenie_z_życia_0_do_100 ~ Wiek * Typ_zamieszkania ===


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '


,sum_sq,df,F,PR(>F)
C(Wiek),2.819267,2.0,0.059244,0.808237
C(Typ_zamieszkania),46.289264,2.0,0.972723,0.326589
C(Wiek):C(Typ_zamieszkania),5.611924,4.0,0.058964,0.981110
Residual,2189.016434,92.0,NaN,NaN


# 8. ANALIZA POWTARZANYCH POMIARÓW (4 miesiące: styczeń, luty, marzec, kwiecień) – pyt.11,12

In [38]:
print("\n=== REPEATED MEASURES ANOVA: Zadowolenie przez 4 miesiące ===")

df_long_rm = pd.melt(
    df,
    id_vars='Lp',  # identyfikator badanych
    value_vars=['Zadowolenie__styczeń', 'Zadowolenie__luty',
                'Zadowolenie__marzec', 'Zadowolenie__kwiecień'],
    var_name='Miesiąc',
    value_name='Zadowolenie'
)

# ANOVA z powtarzanymi pomiarami (statsmodels)
rm_model = AnovaRM(
    data=df_long_rm,
    depvar='Zadowolenie',
    subject='Lp',
    within=['Miesiąc']
)
rm_res = rm_model.fit()
print(rm_res)

# Alternatywnie w pingouin:
rm_anova = pg.rm_anova(dv='Zadowolenie', within='Miesiąc', subject='Lp', data=df_long_rm, detailed=True)
print("\n=== pingouin.rm_anova wynik ===")
display(rm_anova)

# Post-hoc (parametryczne):
print("\n=== Post-hoc parametryczny (pairwise t-tests z korekcją) ===")
posthoc_rm = pg.pairwise_ttests(
    dv='Zadowolenie', within='Miesiąc', subject='Lp', data=df_long_rm,
    padjust='bonf', # lub holm, fdr_bh, etc.
    parametric=True
)
display(posthoc_rm)

# Jeżeli założenia sferyczności lub normalności były naruszone, pyt.11-12 wskazuje, że
# test Friedmana i post-hoc Durbin–Conover:
print("\n=== TEST FRIEDMANA (Nieparametryczna alternatywa) ===")
fried = pg.friedman(dv='Zadowolenie', within='Miesiąc', subject='Lp', data=df_long_rm)
display(fried)

print("\n=== Post-hoc Durbin-Conover (jeżeli Friedman istotny) ===")
durbin_conover = pg.pairwise_tests(
    dv='Zadowolenie', within='Miesiąc', subject='Lp', data=df_long_rm,
    parametric=False,
    padjust='bonf'
)
display(durbin_conover)


=== REPEATED MEASURES ANOVA: Zadowolenie przez 4 miesiące ===
                Anova
        F Value Num DF  Den DF  Pr > F
--------------------------------------
Miesiąc 19.4046 3.0000 297.0000 0.0000


=== pingouin.rm_anova wynik ===


,Source,SS,DF,MS,F,p-unc,ng2,eps
0,Miesiąc,3687.38,3,1229.126667,19.40456,1.623772e-11,0.128896,0.949512
1,Error,18812.62,297,63.342155,NaN,NaN,NaN,NaN



=== Post-hoc parametryczny (pairwise t-tests z korekcją) ===


/usr/local/lib/python3.11/dist-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Miesiąc,Zadowolenie__kwiecień,Zadowolenie__luty,True,True,6.083527,99.0,two-sided,2.223888e-08,1.334333e-07,bonf,5.146e+05,0.830521
1,Miesiąc,Zadowolenie__kwiecień,Zadowolenie__marzec,True,True,0.209155,99.0,two-sided,8.347574e-01,1.000000e+00,bonf,0.113,0.030364
2,Miesiąc,Zadowolenie__kwiecień,Zadowolenie__styczeń,True,True,5.970919,99.0,two-sided,3.700764e-08,2.220458e-07,bonf,3.165e+05,0.828956
3,Miesiąc,Zadowolenie__luty,Zadowolenie__marzec,True,True,-5.301791,99.0,two-sided,6.980396e-07,4.188237e-06,bonf,1.939e+04,-0.707021
4,Miesiąc,Zadowolenie__luty,Zadowolenie__styczeń,True,True,-0.059011,99.0,two-sided,9.530624e-01,1.000000e+00,bonf,0.111,-0.008400
5,Miesiąc,Zadowolenie__marzec,Zadowolenie__styczeń,True,True,4.677313,99.0,two-sided,9.213082e-06,5.527849e-05,bonf,1701.386,0.703917



=== TEST FRIEDMANA (Nieparametryczna alternatywa) ===


,Source,W,ddof1,Q,p-unc
Friedman,Miesiąc,0.133712,3,40.113497,1.008087e-08



=== Post-hoc Durbin-Conover (jeżeli Friedman istotny) ===


,Contrast,A,B,Paired,Parametric,W-val,alternative,p-unc,p-corr,p-adjust,hedges
0,Miesiąc,Zadowolenie__kwiecień,Zadowolenie__luty,True,False,942.0,two-sided,2.438039e-07,1.462823e-06,bonf,0.830521
1,Miesiąc,Zadowolenie__kwiecień,Zadowolenie__marzec,True,False,2293.5,two-sided,7.664437e-01,1.000000e+00,bonf,0.030364
2,Miesiąc,Zadowolenie__kwiecień,Zadowolenie__styczeń,True,False,856.5,two-sided,1.264019e-07,7.584114e-07,bonf,0.828956
3,Miesiąc,Zadowolenie__luty,Zadowolenie__marzec,True,False,889.5,two-sided,1.143913e-06,6.863475e-06,bonf,-0.707021
4,Miesiąc,Zadowolenie__luty,Zadowolenie__styczeń,True,False,2357.0,two-sided,8.094969e-01,1.000000e+00,bonf,-0.008400
5,Miesiąc,Zadowolenie__marzec,Zadowolenie__styczeń,True,False,1237.0,two-sided,1.556016e-05,9.336094e-05,bonf,0.703917


# 9. ANALIZA MIESZANA (pyt.13,14,15): 4 pomiary vs. typ zamieszkania i status związku

In [51]:
print("\n=== ANALIZA MIESZANA (Mixed ANOVA): 4 pomiary vs. typ zamieszkania & status związku ===")

# Łączenie 'Typ_zamieszkania' i 'Status_związku' w jeden nowy czynnik
df_long_mixed['Typ_Status'] = df_long_mixed['Typ_zamieszkania'].astype(str) + '_' + df_long_mixed['Status_związku'].astype(str)

# Wykonanie Mixed ANOVA z jednym czynnikiem międzygrupowym ('Typ_Status') i jednym wewnątrzgrupowym ('Miesiąc')
mix_res = pg.mixed_anova(
    dv='Zadowolenie',
    within='Miesiąc',
    between='Typ_Status',
    subject='Lp',
    data=df_long_mixed,
    correction=True,    # automatyczna poprawka sferyczności
    effsize='np2'
)

display(mix_res)



=== ANALIZA MIESZANA (Mixed ANOVA): 4 pomiary vs. typ zamieszkania & status związku ===


,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,Typ_Status,200.730571,8,91,25.091321,0.386567,9.252507e-01,NaN,0.032867,NaN,NaN,NaN,NaN
1,Miesiąc,3687.380000,3,273,1229.126667,19.402735,1.964640e-11,4.762674e-11,0.175745,0.949512,True,0.921835,0.158833
2,Interaction,1518.584782,24,273,63.274366,0.998836,4.679213e-01,NaN,0.080722,NaN,NaN,NaN,NaN


# 10. REGRESJA (pyt.16-21)

In [40]:
print("\n=== REGRESJA 1: Ocena_zadowolenia_po_czterech_miesiącach ~ Zadowolenie__styczeń ===")

model_reg1 = smf.ols('Ocena_zadowolenia_po_czterech_miesiącach ~ Zadowolenie__styczeń', data=df).fit()
print(model_reg1.summary())

# R-squared to R^2, a R to pierwiastek z R^2.
r_value = np.sqrt(model_reg1.rsquared)
print(f"R (pierwiastek z R^2) = {r_value:.3f}")


=== REGRESJA 1: Ocena_zadowolenia_po_czterech_miesiącach ~ Zadowolenie__styczeń ===
                                       OLS Regression Results                                       
Dep. Variable:     Ocena_zadowolenia_po_czterech_miesiącach   R-squared:                       0.072
Model:                                                  OLS   Adj. R-squared:                  0.063
Method:                                       Least Squares   F-statistic:                     7.629
Date:                                      Sat, 25 Jan 2025   Prob (F-statistic):            0.00686
Time:                                              18:51:08   Log-Likelihood:                -297.53
No. Observations:                                       100   AIC:                             599.1
Df Residuals:                                            98   BIC:                             604.3
Df Model:                                                 1                                         
Covari

In [41]:
# 10.2. Tworzenie zmiennej binarnej: posiadanie_zwierzęcia_bin (tak/nie)
def zwierze_binarne(x):
    x = str(x).lower().strip()
    if 'pies' in x or 'kot' in x:
        return 'tak'
    else:
        return 'nie'

df['Posiadanie_zwierzęcia_bin'] = df['Posiadanie_zwierzęcia'].apply(zwierze_binarne)

print("\nWartości zmiennej binarnej Posiadanie_zwierzęcia_bin:")
print(df['Posiadanie_zwierzęcia_bin'].value_counts())


Wartości zmiennej binarnej Posiadanie_zwierzęcia_bin:
Posiadanie_zwierzęcia_bin
tak    55
nie    45
Name: count, dtype: int64


In [43]:
# 10.3. Wielokrotna regresja: Zależna = Ocena_zadowolenia_po_czterech_miesiącach
# Predyktory: 4 oceny poszczególnych miesięcy + posiadanie_zwierzęcia_bin
print("\n=== REGRESJA 2: Ocena_zadowolenia_po_czterech_miesiącach ~ 4xZadowolenie + binarne zwierzę ===")

formula_reg2 = """
Ocena_zadowolenia_po_czterech_miesiącach
~ Zadowolenie__styczeń
+ Zadowolenie__luty
+ Zadowolenie__marzec
+ Zadowolenie__kwiecień
+ C(Posiadanie_zwierzęcia_bin)
"""
model_reg2 = smf.ols(formula_reg2, data=df).fit()
print(model_reg2.summary())

# Sprawdzenie założeń: normalność reszt, Durbin-Watson, VIF
res = model_reg2.resid
shap_reg2 = pg.normality(res, method='shapiro')
dw = sm.stats.stattools.durbin_watson(res)

print("\nTest normalności reszt (Shapiro-Wilk) dla modelu_reg2:")
display(shap_reg2)
print(f"Durbin-Watson = {dw:.2f}")




=== REGRESJA 2: Ocena_zadowolenia_po_czterech_miesiącach ~ 4xZadowolenie + binarne zwierzę ===
                                       OLS Regression Results                                       
Dep. Variable:     Ocena_zadowolenia_po_czterech_miesiącach   R-squared:                       0.467
Model:                                                  OLS   Adj. R-squared:                  0.439
Method:                                       Least Squares   F-statistic:                     16.48
Date:                                      Sat, 25 Jan 2025   Prob (F-statistic):           1.18e-11
Time:                                              18:52:19   Log-Likelihood:                -269.81
No. Observations:                                       100   AIC:                             551.6
Df Residuals:                                            94   BIC:                             567.2
Df Model:                                                 5                                     

,W,pval,normal
0,0.991318,0.769916,True


Durbin-Watson = 1.91


In [61]:
# VIF:
X_for_vif = df[['Zadowolenie__styczeń', 'Zadowolenie__luty', 'Zadowolenie__marzec', 'Zadowolenie__kwiecień']].copy()
X_for_vif['Zwierze_bin_tak'] = pd.get_dummies(df['Posiadanie_zwierzęcia_bin'], drop_first=True)
X_for_vif = sm.add_constant(X_for_vif)
X_for_vif = X_for_vif.astype(float)

# Obliczenie VIF dla każdej zmiennej
vif_data = []
for i, col in enumerate(X_for_vif.columns):
    v = variance_inflation_factor(X_for_vif.values, i)
    vif_data.append((col, v))

# Wyświetlenie wyników VIF
print("\nVIF:")
for col, v in vif_data:
    print(f"{col}: {v:.3f}")


VIF:
const: 253.587
Zadowolenie__styczeń: 1.021
Zadowolenie__luty: 1.022
Zadowolenie__marzec: 1.038
Zadowolenie__kwiecień: 1.030
Zwierze_bin_tak: 1.030


In [45]:
# 10.4. Regresja krokowa (pyt.21: najpierw sam zwierzę, potem 4 oceny)
print("\n=== REGRESJA KROKOWA (DWUKROTNIE) ===")

# Krok 1: tylko Posiadanie_zwierzęcia_bin
model_step1 = smf.ols(
    'Ocena_zadowolenia_po_czterech_miesiącach ~ C(Posiadanie_zwierzęcia_bin)',
    data=df
).fit()
print("=== Model krok 1: tylko zwierzę_bin ===")
print(model_step1.summary())

# Krok 2: zwierzę_bin + 4 oceny
model_step2 = smf.ols(
    'Ocena_zadowolenia_po_czterech_miesiącach ~ C(Posiadanie_zwierzęcia_bin) + Zadowolenie__styczeń + Zadowolenie__luty + Zadowolenie__marzec + Zadowolenie__kwiecień',
    data=df
).fit()
print("\n=== Model krok 2: zwierzę_bin + 4 oceny ===")
print(model_step2.summary())




=== REGRESJA KROKOWA (DWUKROTNIE) ===
=== Model krok 1: tylko zwierzę_bin ===
                                       OLS Regression Results                                       
Dep. Variable:     Ocena_zadowolenia_po_czterech_miesiącach   R-squared:                       0.001
Model:                                                  OLS   Adj. R-squared:                 -0.009
Method:                                       Least Squares   F-statistic:                    0.1224
Date:                                      Sat, 25 Jan 2025   Prob (F-statistic):              0.727
Time:                                              18:52:55   Log-Likelihood:                -301.22
No. Observations:                                       100   AIC:                             606.4
Df Residuals:                                            98   BIC:                             611.6
Df Model:                                                 1                                         
Covariance T

# Podsumowanie analizy

## Najważniejsze wyniki

- **Zmienność w czasie**: Poziom zadowolenia z życia zmieniał się istotnie między czterema miesiącami, co zostało potwierdzone zarówno ANOVA powtarzanych pomiarów, jak i testem Friedmana. Szczegółowe testy post-hoc ujawniły różnice między poszczególnymi miesiącami.
- **Wpływ czynników społecznych i demograficznych**:
  - Typ zamieszkania oraz status związku nie miały istotnego wpływu na poziom zadowolenia.
  - Posiadanie zwierzęcia domowego nie wpływało znacząco na ogólne zadowolenie z życia.
- **Predykcja poziomu zadowolenia**:
  - Modele regresji wskazały, że oceny zadowolenia z poszczególnych miesięcy były silnymi predyktorami ogólnego zadowolenia po czterech miesiącach.
  - Posiadanie zwierzęcia domowego nie było istotnym predyktorem w żadnym modelu.

## Kluczowe wnioski

1. **Czas jako kluczowy czynnik**: Poziom zadowolenia z życia zmienia się istotnie w czasie, niezależnie od czynników takich jak typ zamieszkania czy status związku.
2. **Rola ocen miesięcznych**: Oceny miesięczne są silnymi wskaźnikami ogólnego zadowolenia po czterech miesiącach, co sugeruje, że codzienne doświadczenia mają większy wpływ na subiektywne postrzeganie zadowolenia niż stałe czynniki demograficzne.
3. **Założenia statystyczne**:
   - Modele regresji oraz analizy ANOVA spełniły wymagane założenia, co zwiększa wiarygodność wyników.

## Perspektywy dalszych badań

W przyszłości warto uwzględnić:
- Inne czynniki mogące wpływać na zadowolenie z życia, takie jak poziom dochodów, aktywność fizyczna czy wsparcie społeczne.
- Analizy bardziej zaawansowane, np. modele mieszane z efektami losowymi.

Dzięki kompleksowej analizie uzyskano wartościowe informacje, które mogą być podstawą do dalszych badań w zakresie psychologii społecznej i analizy zachowań ludzkich.
